In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy
from PIL import Image
import io

In [2]:
K = 64
batch_size = 64
device = 'cuda'

In [3]:
class OM_Encoder(nn.Module):
    def __init__(self):
        super(OM_Encoder,self).__init__()
        self.fc1 = nn.Linear(3,128)
        self.fc2 = nn.Linear(128,128)
        self.fc3 = nn.Linear(256,128)
        self.fc4 = nn.Linear(256,128)
        self.mean_fc = nn.Linear(128,128)
        self.logstddev_fc = nn.Linear(128,128)
    
    def forward(self,x):
        x = x.squeeze()
        n, c, k = x.size()
        x = x.permute(0,2,1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        n,k,c = x.size()
        x = x.permute(0,2,1)
        pooled = F.max_pool1d(x, k).expand(x.size())
        x = torch.cat([x,pooled],dim=1)
        x = x.permute(0,2,1)

        x = F.relu(x)
        x = self.fc3(x)
        n,k,c = x.size()
        x = x.permute(0,2,1)
        pooled = F.max_pool1d(x, k)
        pooled = pooled.expand(x.size())
        x = torch.cat([x,pooled],dim=1)
        x = x.permute(0,2,1)

        x = F.relu(x)
        x = self.fc4(x)
        n,k,c = x.size()
        x = x.permute(0,2,1)
        x = F.max_pool1d(x, k)
        x= x.squeeze()

        mean = self.mean_fc(x)
        stddev = self.logstddev_fc(x)
        
        return mean,stddev

In [4]:
class Block(nn.Module):
    def __init__(self):
        super(Block,self).__init__()
        self.fc1 = nn.Conv2d(256,256,kernel_size=1)
        self.fc2 = nn.Conv2d(256,256,kernel_size=1)
        self.bn1 = nn.BatchNorm2d(256, affine=False, track_running_stats=True)
        self.bn2 = nn.BatchNorm2d(256, affine=False, track_running_stats=True)
        self.gammaLayer1 = nn.Conv1d(128,256,kernel_size=1)
        self.gammaLayer2 = nn.Conv1d(128,256,kernel_size=1)
        self.betaLayer1 = nn.Conv1d(128,256,kernel_size=1)
        self.betaLayer2 = nn.Conv1d(128,256,kernel_size=1)
        
    def forward(self,y):
        x = y['ex']
        n,c,k,d = x.size()

        encoding = y['enc']
        gamma = self.gammaLayer1(encoding)

        #Need to stack the beta and gamma
        #so that we multiply all the points for one mesh
        #by the same value
        gamma = torch.stack([gamma for _ in range(k)],dim=2)
        
        beta = self.betaLayer1(encoding)
        beta = torch.stack([beta for _ in range(k)],dim=2)

        #First apply Conditional Batch Normalization
        out = gamma*self.bn1(x) + beta
        #Then ReLU activation function
        out = F.relu(out)
        #fully connected layer
        out = self.fc1(out)
        #Second CBN layer
        gamma = self.gammaLayer2(encoding)
        gamma = torch.stack([gamma for _ in range(k)],dim=2)

        beta = self.betaLayer2(encoding)
        beta = torch.stack([beta for _ in range(k)],dim=2)
        
        out = gamma* self.bn2(out) + beta
        #RELU activation
        out = F.relu(out)
        #2nd fully connected
        out = self.fc2(out)
        #Add to the input of the ResNet Block 
        out = x + out
        
        return {'ex':out, 'enc':encoding}

In [5]:
class OccupancyModel(nn.Module):
    def __init__(self):
        super(OccupancyModel,self).__init__()
        self.blocks = self.makeBlocks()
        self.encoderModel = OM_Encoder()
        self.gammaLayer = nn.Conv1d(128,256,kernel_size=1)
        self.betaLayer = nn.Conv1d(128,256,kernel_size=1)
        self.cbn = nn.BatchNorm2d(256, affine=False, track_running_stats=True)
        self.fc1 = nn.Conv2d(3,256,kernel_size=1)
        self.fc2 = nn.Conv2d(256,1,kernel_size=1)
        
    def makeBlocks(self):
        blocks = []
        for _ in range(5):
            blocks.append(Block())
        return nn.Sequential(*blocks)
   
    def sampleFromZDist(self, z):
        mean, logstddev = z
        std = logstddev.mul(0.5).exp_()
        eps = torch.randn_like(logstddev,requires_grad=True)
        return eps.mul(std).add_(mean)
        
    def forward(self,x, z_eval=None):
        if self.training:
            z_dist = self.encoderModel(x)
            z = self.sampleFromZDist(z_dist)
            z = z.unsqueeze(-1)
        else:
            z = z_eval
            z_dist = (0,1)
        x = self.fc1(x)
        #5 pre-activation ResNet-blocks
        x = self.blocks({'enc':z, 'ex':x })
        x = x['ex']
        n,c,k,d = x.size()
        
        #CBN
        gamma = self.gammaLayer(z)
        
        gamma = torch.stack([gamma for _ in range(k)],dim=2)
        
        beta = self.betaLayer(z)
        beta = torch.stack([beta for _ in range(k)],dim=2)

        x = gamma.mul(self.cbn(x)).add_(beta)
        x = F.relu(x)
        x = self.fc2(x)
        #x = x.view(-1,1)
        #x = torch.sigmoid(x)
        return x, z_dist

In [ ]:
#choose a category and load all of the available data:
import random
topdir = "/home/andrea/Documents/GradSchool/OccupancyNetworks/occupancy_networks"

#One DataSetClass per subdirectory in a category, will return "K" point samples and a single image randomly
#drawn from the 23 available
class DataSetClass(torch.utils.data.Dataset):
    def __init__(self, d):
        self.dir = d
        with numpy.load(f"{d}/points.npz") as data:
            self.pts = torch.tensor(data["points"], dtype=torch.float)
            self.occupancies = torch.tensor(numpy.unpackbits(data["occupancies"])[:self.pts.size()[0]], dtype=torch.float)
        self.K = K 
        self.length = int(self.occupancies.size()[0]/self.K)
    def __len__(self):
        return self.length
    
    def __getitem__(self,idx):
        return self.pts[idx*self.K:(idx*self.K+self.K)], self.occupancies[idx*self.K:(idx*self.K+self.K)]

       
#catalogue all of the directories with the chosen category
trainingDirs = []
couchesDirectory=f"{topdir}/data/ShapeNet/04256520"

#Get the test data
testDirs = []
with io.open(f"{couchesDirectory}/test.lst") as testlist:
    for testdir in testlist.readlines():
        testDirs.append(f"{couchesDirectory}/{testdir.strip()}")
dataSets = []
for tdir in testDirs:
    dataSets.append(DataSetClass(tdir))
test_data = torch.utils.data.ConcatDataset(dataSets)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

In [6]:
def generateGrid(ncuts, xl, xr, yl, yr, zl, zr):
    #The unit cube centered at 0
    #Subdivided into a grid of 32^3 "voxels"
    x = numpy.linspace(xl,xr,ncuts)
    y = numpy.linspace(yl,yr,ncuts)
    z = numpy.linspace(zl,zr,ncuts)
    xg,yg,zg = numpy.meshgrid(x,y,z)
    x = torch.tensor(xg)
    y = torch.tensor(yg)
    z = torch.tensor(zg)
    #Convert to a grid of 3 dimensional coordinate
    tgrid = torch.stack([x,y,z], dim=3).permute(1,0,2,3)
    #A cube is made up the 8 vertices
    #Convert to a list where every 8 coords denote a cube
    gridpts = torch.zeros(8*(ncuts-1)*(ncuts-1)*(ncuts-1),3)

    '''
    Vertex Order for marching cubes is 
    (0,0,0):(1,0,0):(1,1,0):(0,1,0):(0,0,1):(1,0,1):(1,1,1):(0,1,1)
    '''
    gpt = 0
    for i in range(ncuts-1):
        for j in range(ncuts-1):
            for k in range(ncuts-1):
                gridpts[gpt] = tgrid[i][j][k]
                gridpts[gpt+1] = tgrid[i+1][j][k]
                gridpts[gpt+2] = tgrid[i+1][j+1][k]
                gridpts[gpt+3] = tgrid[i][j+1][k]
                gridpts[gpt+4] = tgrid[i][j][k+1]
                gridpts[gpt+5] = tgrid[i+1][j][k+1]
                gridpts[gpt+6] = tgrid[i+1][j+1][k+1]
                gridpts[gpt+7] = tgrid[i][j+1][k+1]
                gpt = gpt + 8

    return gridpts

In [7]:
def generateAdaptiveGrid(ncuts, xl, xh, yl, yh, zl, zh, limit,meshFunct, onCuda):
    if not limit:
        return None 
    g = generateGrid(ncuts, xl, xh, yl, yh, zl, zh)
    ag = g.view(-1,3)
    
    finalGrid = []
    #divide list of coordinates into cubes
    for i in range(0,int(ag.size()[0]),8):
        #marking one active if occupancies differ on the vertices 
        active = False
        for k in range(0,8):
            coord = ag[i + k]
            if onCuda:
                coord = coord.cuda()
            active ^= meshFunct(coord)
        if(active):
            #near left coordinate is v0
            nl = ag[i]
            #top right coordinate is v6
            tr = ag[i + 6]

            #subdivide this cube into 8 subvoxels
            g = generateAdaptiveGrid(3,nl[0],tr[0], nl[1],tr[1],nl[2],tr[2], limit-1, meshFunct, onCuda)
            #replace this grid where the cube was 
            if g is not None:
                finalGrid.append(g)
            #or keep this cube
            else:
                finalGrid.append(ag[i: i + 8])
        else:
            finalGrid.append(ag[i:i+8])
    for i in range(len(finalGrid)):
        finalGrid[i] = finalGrid[i].view(-1,3)
    finalGrid = torch.cat(finalGrid)

    return finalGrid

In [8]:
from functools import partial
def overModelThreshold(model, pt,z):
    x = model(pt.view(1,3,1),z)
    return (x > 0.5).item()


In [9]:
## Evaluate the model on an adaptive Grid
model = OccupancyModel()
model.load_state_dict(torch.load("../training/unconditional_model3.pth",map_location=device))
model.cuda()
model.eval()
f = partial(overModelThreshold,model)
#g = generateAdaptiveGrid(32,-0.5,0.5,-0.5,0.5,-0.5,0.5,3, f, True)
#numpy.savetxt('ag_32_3.txt', g.detach().numpy())
pts = torch.tensor(numpy.loadtxt('bench_ag_32_3.txt'), dtype=torch.float)




In [17]:
from torch.autograd import Variable
z = Variable(torch.randn(1,128))
z= z.unsqueeze(-1).cuda()
for i in range(10):
    #how to interpolate? add .1 to each dimension? 
    z.add_(0.1)
    occ = []
    with torch.no_grad():
        for p in pts:
            c = p.view(1,3,1).cuda()
            c = c.unsqueeze(-1)
            pred, z_dist = model(c,z)
            occ.append(torch.sigmoid(pred).cpu())
    numpy.savetxt(f'couch_interp/couch_ag_preds_32_{i}.txt', occ)